# Example 5: Special Token Mask Detection

This notebook demonstrates how to:
1. Load a language model
2. Create `ModelInputDetector` with special token mask detection enabled
3. Use model's automatic special token detection OR provide custom special token IDs
4. Run inference and capture special token masks
5. Visualize and verify the mask correctness
6. Save the mask to store

The special token mask is a binary mask (1 for special tokens, 0 for regular tokens) that has the same shape as `input_ids`. This is useful for:
- Filtering out special tokens during analysis
- Understanding tokenization behavior
- Creating attention masks that exclude special tokens
- Analyzing model behavior on special vs regular tokens


In [1]:
%load_ext autoreload
%autoreload 2

import torch
from pathlib import Path
import numpy as np

from amber.hooks import ModelInputDetector
from amber.language_model.language_model import LanguageModel
from amber.store.local_store import LocalStore

print("✅ Imports completed")


/Users/adam/Projects/Inzynierka/codebase/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ Imports completed


In [2]:
# MODEL_ID = "sshleifer/tiny-gpt2"
MODEL_ID = "speakleash/Bielik-1.5B-v3.0-Instruct"
STORE_DIR = Path("store")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

TEST_TEXTS = [
    "Hello world! This is a test.",
    "The quick brown fox jumps over the lazy dog.",
    "Machine learning is fascinating.",
]

print("🚀 Starting Special Token Mask Example")
print(f"📱 Using device: {DEVICE}")
print(f"🔧 Model: {MODEL_ID}")
print(f"📝 Number of test texts: {len(TEST_TEXTS)}")
print()

STORE_DIR.mkdir(parents=True, exist_ok=True)
print("✅ Output directories created")


🚀 Starting Special Token Mask Example
📱 Using device: cpu
🔧 Model: speakleash/Bielik-1.5B-v3.0-Instruct
📝 Number of test texts: 3

✅ Output directories created


In [3]:
print("📥 Loading language model...")

store = LocalStore(STORE_DIR)
lm = LanguageModel.from_huggingface(MODEL_ID, store=store)
lm.model.to(DEVICE)

print(f"✅ Model loaded: {lm.model_id}")
print(f"📱 Device: {DEVICE}")
print(f"📁 Store location: {lm.context.store.base_path}")
print()

tokenizer = lm.tokenizer
print("🔍 Tokenizer special tokens:")
special_token_attrs = ['pad_token_id', 'eos_token_id', 'bos_token_id', 'unk_token_id', 
                       'cls_token_id', 'sep_token_id', 'mask_token_id']
special_tokens = {}
for attr in special_token_attrs:
    token_id = getattr(tokenizer, attr, None)
    if token_id is not None:
        special_tokens[attr] = token_id
        print(f"  {attr}: {token_id}")

if hasattr(tokenizer, 'all_special_ids'):
    print(f"  all_special_ids: {tokenizer.all_special_ids}")


📥 Loading language model...
✅ Model loaded: speakleash_Bielik-1.5B-v3.0-Instruct
📱 Device: cpu
📁 Store location: store

🔍 Tokenizer special tokens:
  pad_token_id: 2
  eos_token_id: 4
  bos_token_id: 1
  unk_token_id: 0
  all_special_ids: [1, 4, 0, 2, 3, 5, 6]


## Debug

At the top of the demo we check token_ids in lm.tokenizer
```
tokenizer = lm.tokenizer
...
token_id = getattr(tokenizer, attr, None)
```

Then in ModelInputDetector we pass lm.model

```
lm.layers.name_to_layer[layer_signature] = lm.model
```

and then in _get_special_token_ids check for module.tokenizer (does not exist) and module.config (exists but sometimes contains lists as token_ids)

In [4]:
module = lm.model
if hasattr(module, 'config'):
    config = module.config
    token_id_attrs = ['pad_token_id', 'eos_token_id', 'bos_token_id', 'unk_token_id',
                            'cls_token_id', 'sep_token_id', 'mask_token_id']
    for attr in token_id_attrs:
        token_id = getattr(config, attr, None)
        if token_id is not None:
            print(attr)
            print(token_id)

pad_token_id
2
eos_token_id
[4, 2]
bos_token_id
1


In [5]:
if hasattr(module, 'tokenizer'):
    tokenizer = module.tokenizer
    token_id_attrs = ['pad_token_id', 'eos_token_id', 'bos_token_id', 'unk_token_id', 
                    'cls_token_id', 'sep_token_id', 'mask_token_id']
    for attr in token_id_attrs:
        token_id = getattr(tokenizer, attr, None)
        if token_id is not None:
            print(attr)
            print(token_id)

## Option 1: Auto-detect Special Tokens from Model

The detector will automatically extract special token IDs from the model's tokenizer or config.


In [6]:
print("🔧 Creating ModelInputDetector with auto-detection of special tokens...")

layer_signature = "model_inputs_with_mask"
if layer_signature not in lm.layers.name_to_layer:
    lm.layers.name_to_layer[layer_signature] = lm.model
    print(f"📝 Added '{layer_signature}' to layers registry")

input_detector = ModelInputDetector(
    layer_signature=layer_signature,
    hook_id="model_input_detector_with_mask",
    save_input_ids=True,
    save_attention_mask=False,
    save_special_token_mask=True,
    special_token_ids=None,
)

hook_id = lm.layers.register_hook(layer_signature, input_detector)

print(f"✅ Detector attached to model")
print(f"🆔 Detector ID: {input_detector.id}")
print(f"💾 Will save: input_ids, special_token_mask")


🔧 Creating ModelInputDetector with auto-detection of special tokens...
📝 Added 'model_inputs_with_mask' to layers registry
✅ Detector attached to model
🆔 Detector ID: model_input_detector_with_mask
💾 Will save: input_ids, special_token_mask


In [ ]:
print("🚀 Running inference...")
print(f"📝 Processing {len(TEST_TEXTS)} texts")

input_detector.clear_captured()

output, encodings = lm.forwards(
    TEST_TEXTS,
    tok_kwargs={"max_length": 128, "padding": True, "truncation": True, "add_special_tokens": True},
    autocast=False,
)

print("✅ Inference completed")
print(f"📊 Encodings keys: {list(encodings.keys())}")
print()
print("💡 Data captured in detector - ready to inspect")


🚀 Running inference...
📝 Processing 3 texts
✅ Inference completed
📊 Encodings keys: ['input_ids', 'attention_mask']

💡 Data captured in detector - ready to inspect


In [18]:
encodings

{'input_ids': tensor([[    2,     2,     2,     2,     2,     2,     2,     2,     1, 10404,
            397, 22299, 31964, 22382,  3707,   322,  6291, 31917],
         [    1,  2091,  9108, 23156, 31225, 31892,  2228, 31967,   590,  4742,
          31896, 17419,  1226,  1347,   395,   303, 31908, 31917],
         [    2,     2,     2,     2,     2,     1,   739,  1437,   289,   568,
            300,  4957,  3707,  1075, 31896,  5910, 19217, 31917]]),
 'attention_mask': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
         [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}

In [9]:
output.to_tuple()[0].shape

torch.Size([3, 18, 32000])

In [10]:
input_ids = input_detector.get_captured_input_ids()
special_token_mask = input_detector.get_captured_special_token_mask()

print("📊 Captured Data:")
print(f"  input_ids shape: {input_ids.shape}")
print(f"  special_token_mask shape: {special_token_mask.shape}")
print(f"  special_token_mask dtype: {special_token_mask.dtype}")
print()

assert input_ids.shape == special_token_mask.shape, "Shapes must match!"
print("✅ Shapes match!")
print()

print("🔍 Special Token Mask Analysis:")
for i, text in enumerate(TEST_TEXTS):
    print(f"\nText {i+1}: {text[:50]}...")
    print(f"  input_ids: {input_ids[i].tolist()}")
    print(f"  mask:      {special_token_mask[i].int().tolist()}")
    
    num_special = special_token_mask[i].sum().item()
    num_total = len(input_ids[i])
    print(f"  Special tokens: {num_special}/{num_total} ({100*num_special/num_total:.1f}%)")


📊 Captured Data:
  input_ids shape: torch.Size([3, 18])
  special_token_mask shape: torch.Size([3, 18])
  special_token_mask dtype: torch.bool

✅ Shapes match!

🔍 Special Token Mask Analysis:

Text 1: Hello world! This is a test....
  input_ids: [2, 2, 2, 2, 2, 2, 2, 2, 1, 10404, 397, 22299, 31964, 22382, 3707, 322, 6291, 31917]
  mask:      [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Special tokens: 9/18 (50.0%)

Text 2: The quick brown fox jumps over the lazy dog....
  input_ids: [1, 2091, 9108, 23156, 31225, 31892, 2228, 31967, 590, 4742, 31896, 17419, 1226, 1347, 395, 303, 31908, 31917]
  mask:      [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Special tokens: 1/18 (5.6%)

Text 3: Machine learning is fascinating....
  input_ids: [2, 2, 2, 2, 2, 1, 739, 1437, 289, 568, 300, 4957, 3707, 1075, 31896, 5910, 19217, 31917]
  mask:      [1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
  Special tokens: 6/18 (33.3%)


In [11]:
# Step 5: Visualize mask
print("📈 Visualizing Special Token Mask:")
print()

for i, text in enumerate(TEST_TEXTS):
    print(f"Text {i+1}: {text}")
    print(f"  Token IDs:     {input_ids[i].tolist()}")
    print(f"  Special Mask:  {' '.join(['█' if m else '░' for m in special_token_mask[i].tolist()])}")
    print(f"  Values:        {' '.join(['1' if m else '0' for m in special_token_mask[i].tolist()])}")
    print()
    
    # Decode tokens to verify
    token_ids_list = input_ids[i].tolist()
    mask_list = special_token_mask[i].tolist()
    
    print("  Token breakdown:")
    for j, (token_id, is_special) in enumerate(zip(token_ids_list, mask_list)):
        token_str = tokenizer.decode([token_id])
        special_marker = "[SPECIAL]" if is_special else ""
        print(f"    [{j:2d}] ID={token_id:4d} | {token_str:20s} {special_marker}")
    print()


📈 Visualizing Special Token Mask:

Text 1: Hello world! This is a test.
  Token IDs:     [2, 2, 2, 2, 2, 2, 2, 2, 1, 10404, 397, 22299, 31964, 22382, 3707, 322, 6291, 31917]
  Special Mask:  █ █ █ █ █ █ █ █ █ ░ ░ ░ ░ ░ ░ ░ ░ ░
  Values:        1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0

  Token breakdown:
    [ 0] ID=   2 | </s>                 [SPECIAL]
    [ 1] ID=   2 | </s>                 [SPECIAL]
    [ 2] ID=   2 | </s>                 [SPECIAL]
    [ 3] ID=   2 | </s>                 [SPECIAL]
    [ 4] ID=   2 | </s>                 [SPECIAL]
    [ 5] ID=   2 | </s>                 [SPECIAL]
    [ 6] ID=   2 | </s>                 [SPECIAL]
    [ 7] ID=   2 | </s>                 [SPECIAL]
    [ 8] ID=   1 | <s>                  [SPECIAL]
    [ 9] ID=10404 | Hel                  
    [10] ID= 397 | lo                   
    [11] ID=22299 | world                
    [12] ID=31964 | !                    
    [13] ID=22382 | This                 
    [14] ID=3707 | is                   


## Option 2: Provide Custom Special Token IDs

You can also explicitly provide special token IDs instead of auto-detecting.


In [12]:
# Step 6: Create detector with custom special token IDs
print("🔧 Creating ModelInputDetector with custom special token IDs...")

# Example: Only mark pad_token and eos_token as special
custom_special_ids = []
if tokenizer.pad_token_id is not None:
    custom_special_ids.append(tokenizer.pad_token_id)
if tokenizer.eos_token_id is not None:
    custom_special_ids.append(tokenizer.eos_token_id)

print(f"  Custom special token IDs: {custom_special_ids}")

# Create new detector with custom IDs
layer_signature_custom = "model_inputs_custom_mask"
if layer_signature_custom not in lm.layers.name_to_layer:
    lm.layers.name_to_layer[layer_signature_custom] = lm.model

custom_detector = ModelInputDetector(
    layer_signature=layer_signature_custom,
    hook_id="model_input_detector_custom_mask",
    save_input_ids=True,
    save_special_token_mask=True,
    special_token_ids=custom_special_ids,  # Custom special token IDs
)

lm.layers.register_hook(layer_signature_custom, custom_detector)
print("✅ Custom detector created")


🔧 Creating ModelInputDetector with custom special token IDs...
  Custom special token IDs: [2, 4]
✅ Custom detector created


In [13]:
# Step 7: Test custom detector
print("🚀 Testing custom detector...")

custom_detector.clear_captured()

output, encodings = lm.forwards(
    TEST_TEXTS,
    tok_kwargs={"max_length": 128, "padding": True, "truncation": True, "add_special_tokens": True},
    autocast=False,
)

custom_detector.set_inputs_from_encodings(encodings, module=lm.model)

custom_mask = custom_detector.get_captured_special_token_mask()
print("✅ Custom mask captured")
print(f"\n📊 Comparison:")
# print(f"  Auto-detected mask sum: {special_token_mask.sum().item()}")
print(f"  Custom mask sum:        {custom_mask.sum().item()}")
# print(f"\n  Auto-detected mask (first text): {special_token_mask[0].int().tolist()}")
print(f"  Custom mask (first text):         {custom_mask[0].int().tolist()}")

custom_mask[1]


🚀 Testing custom detector...
✅ Custom mask captured

📊 Comparison:
  Custom mask sum:        13
  Custom mask (first text):         [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


tensor([False, False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False])

In [14]:
print(lm.tokenizer.tokenize("The quick brown fox jumps over the lazy dog."))

['▁The', '▁qu', 'ick', '▁brow', 'n', '▁fo', 'x', '▁ju', 'mp', 's', '▁over', '▁the', '▁la', 'zy', '▁do', 'g', '.']


In [15]:
# Step 8: Verify mask correctness
print("✅ Verifying mask correctness...")
print()

# Get special token IDs that were used
special_ids_used = input_detector._get_special_token_ids(lm.model)
print(f"Special token IDs used: {sorted(special_ids_used)}")
print()

# Verify: for each position where mask is True, the token ID should be in special_ids
all_correct = True
for i in range(input_ids.shape[0]):
    for j in range(input_ids.shape[1]):
        token_id = input_ids[i, j].item()
        is_special = special_token_mask[i, j].item()
        
        # Check if mask correctly identifies special tokens
        should_be_special = token_id in special_ids_used
        
        if is_special != should_be_special:
            print(f"❌ Mismatch at position [{i}, {j}]: token_id={token_id}, "
                  f"mask={is_special}, should_be_special={should_be_special}")
            all_correct = False

if all_correct:
    print("✅ All mask values are correct!")
    print(f"   Mask correctly identifies {special_token_mask.sum().item()} special token positions")


✅ Verifying mask correctness...

Special token IDs used: [0, 1, 2, 3, 4, 5, 6]

✅ All mask values are correct!
   Mask correctly identifies 16 special token positions


In [16]:
# Step 9: Save to store (optional)
print("💾 Saving to store...")

# Create a run name
run_name = "special_token_mask_demo"
batch_index = 0

# Save detector metadata
lm.save_detector_metadata(run_name, batch_index)

print(f"✅ Saved to store: {run_name}/batch_{batch_index}")
print(f"📁 Store location: {store.base_path}")
print()
print("🎉 Example completed successfully!")


💾 Saving to store...
✅ Saved to store: special_token_mask_demo/batch_0
📁 Store location: store

🎉 Example completed successfully!
